In [1]:
from itertools import count
import heapq
class State:
    def __init__(self, row, col, grid):
        self.row = row
        self.col = col
        self.grid = grid
        self.n = len(grid)

    def goalTest(self):
        return self.row == self.n - 1 and self.col == self.n - 1

    def moveGen(self):
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1),
                      (1, 1), (-1, -1), (1, -1), (-1, 1)]
        children = []
        for dr, dc in directions:
            new_row, new_col = self.row + dr, self.col + dc
            if 0 <= new_row < self.n and 0 <= new_col < self.n:
                if self.grid[new_row][new_col] == 0:
                    children.append(State(new_row, new_col, self.grid))
        return children

    def __eq__(self, other):
        return self.row == other.row and self.col == other.col

    def __hash__(self):
        return hash((self.row, self.col))

    def __repr__(self):
        return f"({self.row},{self.col})"


In [2]:
def heuristic(state):
    return abs(state.n-1-state.row ) + abs(state.n-1 - state.col)

def euclidean(state):
    return math.sqrt((state.row - state.n-1)**2 + (state.col - state.n-1)**2)


In [3]:
def reconstruct_path(parent, current):
    path = []
    while current is not None:
        path.append(current)
        current = parent[current]
    return list(reversed(path))


In [4]:
def best_first_search(start):
    visited = set()
    pq = []
    counter = count()  # unique tie-breaker
    heapq.heappush(pq, (heuristic(start), next(counter), [start]))

    while pq:
        _, _, path = heapq.heappop(pq)
        current = path[-1]

        if current.goalTest():
            return len(path), [(s.row, s.col) for s in path]

        if current not in visited:
            visited.add(current)
            for child in current.moveGen():
                if child not in visited:
                    heapq.heappush(pq, (heuristic(child), next(counter), path + [child]))
    return None, None

In [5]:
def a_star_search(start):
    visited = set()
    pq = []
    counter = count()
    heapq.heappush(pq, (heuristic(start), next(counter), 0, [start]))

    while pq:
        _, _, g, path = heapq.heappop(pq)
        current = path[-1]

        if current.goalTest():
            return len(path), [(s.row, s.col) for s in path]

        if current not in visited:
            visited.add(current)
            for child in current.moveGen():
                if child not in visited:
                    new_g = g + 1
                    f = new_g + heuristic(child)
                    heapq.heappush(pq, (f, next(counter), new_g, path + [child]))
    return None, None


In [6]:
grids = {
    "Example 1": [[0, 1],
                  [1, 0]],
    "Example 2": [[0, 0, 0],
                  [1, 1, 0],
                  [1, 1, 0]],
    "Example 3": [[1, 0, 0],
                  [1, 1, 0],
                  [1, 1, 0]]
}

for name, grid in grids.items():
    print(f"\n{name}:")
    bfs_len, bfs_path = best_first_search(State(0,0,grid))
    astar_len, astar_path = a_star_search(State(0,0,grid))

    print(f"  Best First Search → Path length: {bfs_len}, Path: {bfs_path}")
    print(f"  A* Search         → Path length: {astar_len}, Path: {astar_path}")



Example 1:
  Best First Search → Path length: 2, Path: [(0, 0), (1, 1)]
  A* Search         → Path length: 2, Path: [(0, 0), (1, 1)]

Example 2:
  Best First Search → Path length: 4, Path: [(0, 0), (0, 1), (1, 2), (2, 2)]
  A* Search         → Path length: 4, Path: [(0, 0), (0, 1), (1, 2), (2, 2)]

Example 3:
  Best First Search → Path length: 4, Path: [(0, 0), (0, 1), (1, 2), (2, 2)]
  A* Search         → Path length: 4, Path: [(0, 0), (0, 1), (1, 2), (2, 2)]
